# What's in an Avocado Toast: A Supply Chain Analysis

You're in London, making an avocado toast, a quick-to-make dish that has soared in popularity on breakfast menus since the 2010s. A simple smashed avocado toast can be made with five ingredients: one ripe avocado, half a lemon, a big pinch of salt flakes, two slices of sourdough bread and a good drizzle of extra virgin olive oil. It's no small feat that most of these ingredients are readily available in grocery stores. 

In this project, you'll conduct a supply chain analysis of three of these ingredients used in an avocado toast, utilizing the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.
(https://world.openfoodfacts.org/)
Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are, fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple categories tags in the `categories_tags` column.
- There is a column in each CSV file called `origins_tags` with strings for country of origin of that item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin, and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

![](avocado_wallpaper.jpeg)

In [50]:
### Read in the avocado data

# Read tab-delimited data
import pandas as pd
avocado = pd.read_csv('data/avocado.csv', sep='\t')

# Subset large DataFrame to include only relevant columns
subset_columns = [ 'code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
avocado = avocado[subset_columns]

# Gather relevant categories data for avocados
with open("data/relevant_avocado_categories.txt", "r") as file:
    relevant_avocado_categories = file.read().splitlines()
    file.close()
    
### Filter avocado data using relevant category tags

# Turn a column of comma-separated tags into a column of lists
avocado['categories_list'] = avocado['categories_tags'].str.split(',')



In [51]:
# Drop rows with null values in a particular column
avocado = avocado.dropna(subset = 'categories_list')

# Filter a DataFrame based on a column of lists
avocado = avocado[avocado['categories_list'].apply(lambda x: any([i for i in x if i in relevant_avocado_categories]))]

### Where do most avocados come from?

# Filter DataFrame for UK data
avocados_uk = avocado[(avocado['countries']=='United Kingdom')]

avocados_uk.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags,categories_list
361,00985833,en,Avacado,650 g,NaN,NaN,Marks & Spencer,marks-spencer,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,Peru,en:peru,"[en:plant-based-foods-and-beverages, en:plant-..."
381,00040464,en,Avocado,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
414,4088600100173,en,Avocado,100 g,NaN,en:mixed-plastic-unknown,Aldi,aldi,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
468,01307351,en,Avacados organic,NaN,NaN,"en:card-tray,en:ldpe-bag",Sainsbury’s SO organic,sainsbury-s-so-organic,"en:plant-based-foods-and-beverages,en:plant-ba...","en:organic,en:eu-organic,en:non-eu-agriculture...",United Kingdom,en:united-kingdom,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
508,5057172125395,en,Just Essentials Avocados,4pack,NaN,en:mixed-plastic-film-packet-to-recycle,Asda,asda,"en:plant-based-foods-and-beverages,en:plant-ba...","en:class-i,en:contains-stones",United Kingdom,en:united-kingdom,Peru,en:peru,"[en:plant-based-foods-and-beverages, en:plant-..."


In [52]:
# Find most common country for avocado origin
avocado_origin = (avocados_uk['origins_tags'].value_counts().index[0])
avocado_origin = avocado_origin.lstrip("en:")


In [53]:
### Create a general function to read and filter data for a particular ingredient, 
###    and return the top origin country for that food item

def read_and_filter_data(filename, relevant_categories):
  df = pd.read_csv('data/' + filename, sep='\t')
  
  # Subset large DataFrame to include only relevant columns
  subset_columns = [ 'code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
  df = df[subset_columns]

  # Split tags into lists
  df['categories_list'] = df['categories_tags'].str.split(',')

  # Drop rows with null categories data
  df = df.dropna(subset = 'categories_list')

  # Filter data for relevant categories
  df = df[df['categories_list'].apply(lambda x: any([i for i in x if i in relevant_categories]))]
    
  # Filter data for the UK
  df_uk = df[(df['countries']=='United Kingdom')]

  # Find top origin country string with the highest count
  top_origin_string = (df_uk['origins_tags'].value_counts().index[0])

  # Clean up top origin country string
  top_origin_country = top_origin_string.lstrip("en:")
  top_origin_country = top_origin_country.replace('-', ' ')

  print(f'**{filename[:-4]} origins**','\n', top_origin_country, '\n')

  print ("Top origin country: ", top_origin_country)
  print ("\n")

  # End of function - return top origin country for this ingredient
  return top_origin_country


# Analyze avocado origins again, this time by calling function
top_avocado_origin = read_and_filter_data('avocado.csv',relevant_avocado_categories)

### Repeat process above with new function for the other 2 ingredients

# Gather relevant categories data for olive oil
with open("data/relevant_olive_oil_categories.txt", "r") as file:
    relevant_olive_oil_categories = file.read().splitlines()
    file.close()

# Call user-defined function on olive_oil.csv
top_olive_oil_origin = read_and_filter_data('olive_oil.csv',relevant_olive_oil_categories)

# Gather relevant categories data for sourdough
with open("data/relevant_sourdough_categories.txt", "r") as file:
    relevant_sourdough_categories = file.read().splitlines()
    file.close()

# Call user-defined function on sourdough.csv
top_sourdough_origin = read_and_filter_data('sourdough.csv',relevant_sourdough_categories)

**avocado origins** 
 peru 

Top origin country:  peru


**olive_oil origins** 
 greece 

Top origin country:  greece


**sourdough origins** 
 united kingdom 

Top origin country:  united kingdom


